In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip show torch

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

import time

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
# Hyperparameter
BATCH_SIZE = 32

In [6]:
transformer = transforms.Compose(transforms=[
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5), std=(0.5))
])

train_dataset = torchvision.datasets.EMNIST(root="./data", split='mnist', train=True, download=True, transform=transformer)
trainloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.EMNIST(root="./data", split='mnist', train=False, download=True, transform=transformer)
testloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

  0%|          | 0/561753746 [00:00<?, ?it/s]

Extracting ./data/EMNIST/raw/gzip.zip to ./data/EMNIST/raw


# Helpers

In [7]:
def train_model(model, criterion, optimizer, trainloader, number_of_epochs=10, device='cpu'):
    steps_per_epoch = len(trainloader)
    model.to(device)

    for epoch in range(number_of_epochs):
        running_loss = 0.0

        for (inputs, labels) in trainloader:
            model.to(device)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # forward pass
            outputs = model(inputs)

            # Calculate loss
            loss = criterion(outputs, labels)

            # backward + optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'[{epoch + 1}] loss: {running_loss / steps_per_epoch:.3f}')

    print('Finished training')


def test_model(model, dataloader):
    n_correct = 0
    n_total = 0

    model.to('cpu')
    model.eval()

    with torch.no_grad():
        for (images, labels) in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            n_total += labels.size(0)
            n_correct += (predicted == labels).sum().item()

            result = n_correct / n_total * 100

    print(f"Accuracy on test set: {result:.1f}%")


def model_size(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (KB):', size/1e3)
    os.remove('temp.p')
    return size

def make_prediction(model, x):
    x = torch.unsqueeze(x, 0)
    x = torch.unsqueeze(x, 0)
    
    model.to('cpu')
    model.eval()
    
    pred = model(x)
    _, pred = torch.max(pred, 1)
    
    return int(pred)

# Basic model

In [8]:
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    # in_channels = 1, because nn working with grayscale images
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3) 
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3) 
    # out_channels = number of filters
    
    self.relu = nn.ReLU()
    self.relu1 = nn.ReLU()
    self.relu2 = nn.ReLU()
    
    self.linear1 = nn.Linear(in_features=16*5*5, out_features = 128) 
    self.linear2 = nn.Linear(in_features=128, out_features=10) 
    # out_features = number of classes

  def forward(self, x):

    x = self.relu1(self.conv1(x))
    x = self.pool(x)

    x = self.relu2(self.conv2(x))
    x = self.pool(x)

    x = torch.flatten(x, 1) 
    x = self.relu(self.linear1(x))
    x = self.linear2(x)
    return x
    
model = ConvNet().to(device)

In [9]:
train_model(
    model=model,
    criterion = nn.CrossEntropyLoss(),
    optimizer=optim.Adam(params=model.parameters(), lr=0.001),
    trainloader=trainloader,
    number_of_epochs=15,
    device='cuda'
)

[1] loss: 0.143
[2] loss: 0.045
[3] loss: 0.033
[4] loss: 0.025
[5] loss: 0.020
[6] loss: 0.016
[7] loss: 0.013
[8] loss: 0.012
[9] loss: 0.009
[10] loss: 0.009
[11] loss: 0.006
[12] loss: 0.007
[13] loss: 0.005
[14] loss: 0.006
[15] loss: 0.005
Finished training


## Accuracy of base model

In [10]:
test_model(
    model=model, 
    dataloader=testloader
)

Accuracy on test set: 99.2%


# Quantized model

In [11]:
class QAT_ConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3) 
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3) 

        self.relu = nn.ReLU()
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()

        
        self.linear1 = nn.Linear(in_features=16*5*5, out_features = 128) 
        self.linear2 = nn.Linear(in_features=128, out_features=10) 
        # out_features = number of classes

        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):

        x = self.quant(x)
        
        x = self.relu1(self.conv1(x))
        x = self.pool(x)

        x = self.relu2(self.conv2(x))
        x = self.pool(x)

        x = torch.flatten(x, 1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        
        x = self.dequant(x)
        return x


# create a model instance
model_fp32 = QAT_ConvNet().to(device)

# model must be set to eval for fusion to work
model_fp32.eval()

# attach a global qconfig, which contains information about what kind
# of observers to attach.
# use default quantization backend
model_fp32.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')

# fuse the activations to preceding layers
model_fp32_fused = torch.quantization.fuse_modules(
    model_fp32, [['conv1', 'relu1']])
model_fp32_fused = torch.quantization.fuse_modules(
    model_fp32_fused, [['conv2', 'relu2']])


# Prepare the model for QAT. 
model_fp32_prepared = torch.quantization.prepare_qat(model_fp32_fused.train())

/opt/conda/lib/python3.7/site-packages/torch/ao/quantization/observer.py:179: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


In [12]:
train_model(
    model=model_fp32_prepared,
    criterion=torch.nn.CrossEntropyLoss(),
    optimizer=optim.Adam(params=model_fp32_prepared.parameters(), lr=0.001),
    trainloader=trainloader,
    number_of_epochs=15,
    device='cuda'
)

/opt/conda/lib/python3.7/site-packages/torch/ao/quantization/fake_quantize.py:325: UserWarning: _aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This warning will only appear once per process. (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/ReduceAllOps.cpp:30.)
  self.is_symmetric_quant,
/opt/conda/lib/python3.7/site-packages/torch/ao/quantization/fake_quantize.py:325: UserWarning: _aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This warning will only appear once per process. (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/TensorCompare.cpp:481.)
  self.is_symmetric_quant,


[1] loss: 0.264
[2] loss: 0.048
[3] loss: 0.035
[4] loss: 0.027
[5] loss: 0.022
[6] loss: 0.018
[7] loss: 0.013
[8] loss: 0.013
[9] loss: 0.011
[10] loss: 0.009
[11] loss: 0.007
[12] loss: 0.008
[13] loss: 0.008
[14] loss: 0.007
[15] loss: 0.004
Finished training


In [13]:
# Convert the observed model to a quantized model. This does several things:
# quantizes the weights, computes and stores the scale and bias value to be
# used with each activation tensor, fuses modules where appropriate,
# and replaces key operators with quantized implementations.

model_fp32_prepared = model_fp32_prepared.to('cpu')
model_fp32_prepared.eval()
model_int8 = torch.quantization.convert(model_fp32_prepared)

## Accuracy of quantized model

In [14]:
test_model(
    model=model_int8, 
    dataloader=testloader
)

Accuracy on test set: 98.9%


# Size of model

In [15]:
print(f"Size of basic model: {model_size(model)}")

model:    	 Size (KB): 232.591
Size of basic model: 232591


In [16]:
print(f"Size of quantized model: {model_size(model_int8)}")

model:    	 Size (KB): 68.439
Size of quantized model: 68439


# Save model

In [17]:
# Save model
torch.save(model.state_dict(), "model_params.pt")
torch.save(model_int8.state_dict(), "model_int8_params.pt")

# Load model
# model = ConvNet()
# model.load_state_dict(torch.load("model_params.pt"))
# model.eval()